# Guide to Bagging Algorithms

## Theoretical Foundations

### The Bootstrap Principle

**Core Idea:** Given a dataset $\mathcal{D} = \{(x_i, y_i)\}_{i=1}^n$, create diverse training sets through resampling to reduce prediction variance.

**Mathematical Formulation:**
For an unstable base learner $h$, bagging constructs an ensemble:

$$F(x) = \frac{1}{M} \sum_{m=1}^M h^{(m)}(x)$$

where each $h^{(m)}$ is trained on bootstrap sample $\mathcal{D}^{(m)}$.

### Bias-Variance Decomposition in Bagging

**Expected Test Error:** For regression with squared loss:

$$\mathbb{E}[(y - \hat{f}(x))^2] = \text{Bias}^2[\hat{f}(x)] + \text{Var}[\hat{f}(x)] + \sigma^2$$

**Bagging's Effect:**
- **Bias:** $\text{Bias}[F(x)] \approx \text{Bias}[h(x)]$ (nearly unchanged)
- **Variance:** $\text{Var}[F(x)] = \rho \text{Var}[h(x)] + \frac{1-\rho}{M}\text{Var}[h(x)]$

where $\rho$ is the correlation between base predictors.

**Key Insight:** Bagging is most effective when:
1. Base learner has high variance (e.g., deep trees)
2. Base predictors have low correlation ($\rho \to 0$)

---

## Bootstrap Sampling Theory

### Classical Bootstrap

**Sampling Process:**
```
ALGORITHM: Bootstrap Sampling
INPUT: Dataset D = {(x_i, y_i)}_{i=1}^n, number of samples M
OUTPUT: M bootstrap datasets

FOR m = 1 to M:
    D^(m) = {}
    FOR i = 1 to n:
        idx = RANDOM_INTEGER(1, n)  // With replacement
        D^(m).ADD(D[idx])
    
RETURN {D^(1), D^(2), ..., D^(M)}
```

**Statistical Properties:**
- Probability a sample appears in bootstrap: $1 - (1 - 1/n)^n \approx 1 - e^{-1} \approx 63.2\%$
- Expected unique samples per bootstrap: $n(1 - e^{-1}) \approx 0.632n$
- Out-of-bag samples: $\approx 0.368n$

### Advanced Bootstrap Variants

#### 1. Balanced Bootstrap
Ensures each original sample appears exactly once across all bootstraps:

$$\sum_{m=1}^M \mathbf{1}[i \in \mathcal{D}^{(m)}] = M \quad \forall i$$

```
ALGORITHM: Balanced Bootstrap
INPUT: Dataset D, M (must divide n evenly)

1. Create M copies of indices [1,2,...,n]
2. Concatenate into single array of size M×n
3. Randomly shuffle the array
4. Split into M chunks of size n
5. Each chunk defines one bootstrap sample
```

#### 2. Subsampling (Subbagging)
Sample $s < n$ points **without replacement**:

**Advantages:**
- Lower computational cost
- Reduced correlation between base learners
- Better for very large datasets

**Variance Analysis:**
For subsample fraction $r = s/n$:

$$\text{Var}[\text{Subbag}] = \left[\rho + \frac{(1-\rho)}{M}\right] \cdot \frac{1}{r} \cdot \text{Var}[h]$$

**Optimal Subsample Rate:**
Theoretical optimum around $r^* \approx 0.632$ (same as bootstrap coverage), but empirically $r \in [0.5, 0.8]$ often works well.

```
ALGORITHM: Subsampling
INPUT: Dataset D, subsample size s, M

FOR m = 1 to M:
    indices = RANDOM_SAMPLE_WITHOUT_REPLACEMENT(n, s)
    D^(m) = D[indices]
```

#### 3. Poisson Bootstrap
Each sample $i$ appears $k_i$ times where $k_i \sim \text{Poisson}(\lambda)$:

**Mathematical Framework:**
- With $\lambda = 1$: $\mathbb{E}[|D^{(m)}|] = n$, $\text{Var}[|D^{(m)}|] = n$
- Limiting distribution matches classical bootstrap
- Easily vectorizable and parallelizable

```
ALGORITHM: Poisson Bootstrap
INPUT: Dataset D, intensity λ = 1, M

FOR m = 1 to M:
    FOR i = 1 to n:
        k_i = POISSON_RANDOM(λ)
        Add (x_i, y_i) to D^(m) exactly k_i times
```

#### 4. m-out-of-n Bootstrap
Bootstrap samples of size $m \neq n$:

**Theory:** Useful when $m = o(n)$ (e.g., $m = \sqrt{n}$ or $m = n^{2/3}$) for:
- Reducing computational cost
- Improving finite-sample properties
- Handling heavy-tailed distributions

**When to Use:**
- Very large datasets ($n > 10^6$)
- Noisy labels with outliers
- Streaming/online learning scenarios

### Bootstrap Sample Complexity

**Effective Sample Size:** The number of unique samples in a bootstrap follows:

$$\mathbb{E}[\text{Unique samples}] = n\left(1 - \left(1 - \frac{1}{n}\right)^n\right)$$

**Asymptotic Distribution:**
As $n \to \infty$, the number of unique samples is approximately:

$$\text{Unique} \sim \mathcal{N}(n(1-e^{-1}), n e^{-1}(1-e^{-1}))$$

---

## Variance Reduction Mathematics

### Correlation-Variance Trade-off

**Ensemble Variance Formula:**
For $M$ predictors with individual variance $\sigma^2$ and pairwise correlation $\rho$:

$$\text{Var}[\bar{h}] = \rho \sigma^2 + \frac{1-\rho}{M} \sigma^2$$

**Decomposition:**
- **Irreducible component:** $\rho \sigma^2$ (correlation-induced floor)
- **Reducible component:** $\frac{1-\rho}{M} \sigma^2$ (decreases with $M$)

### Correlation Sources and Mitigation

#### 1. Data Correlation
Bootstrap samples overlap → correlated predictions

**Mitigation Strategies:**
- Subsampling with lower overlap
- Feature bagging (random subspaces)
- Different training algorithms per tree

#### 2. Model Correlation
Similar model architectures learn similar patterns

**Quantifying Correlation:**
For predictions $\hat{y}_m(x)$ from different models:

$$\rho(x) = \frac{\text{Cov}[\hat{y}_i(x), \hat{y}_j(x)]}{\sqrt{\text{Var}[\hat{y}_i(x)] \text{Var}[\hat{y}_j(x)]}}$$

**Empirical Correlation Estimation:**
```
ALGORITHM: Estimate Prediction Correlation
INPUT: Ensemble {h_m}, test set T

correlations = []
FOR each pair (i,j) where i < j:
    pred_i = [h_i(x) for x in T]
    pred_j = [h_j(x) for x in T]
    corr_ij = PEARSON_CORRELATION(pred_i, pred_j)
    correlations.ADD(corr_ij)

average_correlation = MEAN(correlations)
```

### Optimal Ensemble Size

**Theoretical Minimum Variance:**
As $M \to \infty$:

$$\lim_{M \to \infty} \text{Var}[\bar{h}] = \rho \sigma^2$$

**Practical Considerations:**
- **Plateau effect:** Diminishing returns after variance reduction plateaus
- **Computational budget:** Training time scales linearly with $M$
- **Memory constraints:** Model storage requirements

**Adaptive Stopping Rule:**
```
ALGORITHM: Adaptive Ensemble Size
tolerance = 0.001
window_size = 10
recent_improvements = []

FOR m = window_size+1 to MAX_M:
    current_error = OOB_ERROR(ensemble[:m])
    prev_error = OOB_ERROR(ensemble[:m-1])
    improvement = prev_error - current_error
    
    recent_improvements.ADD(improvement)
    IF LENGTH(recent_improvements) > window_size:
        recent_improvements.POP_FRONT()
    
    avg_improvement = MEAN(recent_improvements)
    IF avg_improvement < tolerance:
        RETURN m
```

---

## Core Bagging Variants

### Algorithm Comparison Matrix

| Variant | Sample Size | Replacement | Overlap | Best Use Case |
|---------|-------------|-------------|---------|---------------|
| **Bootstrap** | $n$ | Yes | ~63% | Standard bagging, moderate $n$ |
| **Subbagging** | $s < n$ | No | $\frac{s}{n}$ | Large datasets, speed priority |
| **Poisson** | $\sim n$ | Weighted | ~63% | Streaming, parallel processing |
| **m-out-of-n** | $m \neq n$ | Yes | Variable | Theory-driven, heavy tails |
| **Balanced** | $n$ | Constrained | Exactly 1 | Minimize sampling variance |

### Subbagging Deep Dive

**Theoretical Analysis:**
For subsample fraction $r = s/n$, the prediction variance becomes:

$$\text{Var}[\text{Subbag}] = \sigma^2 \left[\rho + \frac{1-\rho}{M}\right] \cdot \frac{1}{r}$$

**Bias Introduction:**
Unlike bootstrap, subsampling introduces bias:

$$\text{Bias}[\text{Subbag}] = (1-r) \cdot \text{Bias}_{\text{finite-sample}}$$

**Optimal Trade-off:**
Minimize MSE = Bias² + Variance:

$$r^* = \arg\min_r \left\{ (1-r)^2 \text{Bias}^2 + \frac{\sigma^2[\rho + (1-\rho)/M]}{r} \right\}$$

### Poisson Bootstrap Implementation

**Efficient Vectorized Version:**
```
ALGORITHM: Vectorized Poisson Bootstrap
INPUT: Dataset D, M ensembles

# Generate all Poisson counts at once
counts = POISSON_RANDOM(λ=1, shape=(M, n))

FOR m = 1 to M:
    # Create weighted dataset
    sample_weights = counts[m, :]
    D_weighted = CREATE_WEIGHTED_DATASET(D, sample_weights)
    h_m = TRAIN_WITH_WEIGHTS(D_weighted)
```

**Online/Streaming Adaptation:**
```
ALGORITHM: Online Poisson Bagging
STATE: M online models {h_m}

ON_NEW_SAMPLE (x, y):
    FOR m = 1 to M:
        k = POISSON_RANDOM(λ=1)
        FOR _ in 1 to k:
            h_m.UPDATE(x, y)  # Online learning update
```

---

## Random Forests Deep Dive

### Mathematical Framework

**Enhanced Variance Reduction:**
Random Forests combine two decorrelation mechanisms:
1. **Bootstrap sampling** (data randomness)
2. **Feature bagging** (feature randomness)

**Combined Variance Formula:**
$$\text{Var}[RF] = \left(\rho_{\text{data}} \cdot \rho_{\text{feature}} + \frac{1-\rho_{\text{data}}}{M_{\text{trees}}} + \frac{1-\rho_{\text{feature}}}{M_{\text{try}}}\right) \sigma^2$$

### Feature Selection Strategy

**Theoretical Optimal $m_{try}$:**
For $d$ features, theoretical analysis suggests:

- **Classification:** $m_{try} = \sqrt{d}$
- **Regression:** $m_{try} = d/3$

**Empirical Refinements:**
Based on extensive experiments, better defaults are often:

```
Classification:
├─ Small d (<20): m_try = max(1, √d)  
├─ Medium d (20-1000): m_try = max(2, √d)
└─ Large d (>1000): m_try = max(10, √d/2)

Regression:  
├─ Small d (<20): m_try = max(1, d/3)
├─ Medium d (20-1000): m_try = max(5, d/3)  
└─ Large d (>1000): m_try = max(20, d/5)
```

**Adaptive Feature Selection:**
```
ALGORITHM: Adaptive m_try Selection
INPUT: Dataset D, validation set V

m_try_candidates = [√d/2, √d, 2√d, d/5, d/3, d/2]
best_score = -∞
best_m_try = √d

FOR m_try in m_try_candidates:
    rf = TRAIN_RANDOM_FOREST(D, m_try=m_try, n_trees=100)
    score = EVALUATE(rf, V)
    
    IF score > best_score:
        best_score = score
        best_m_try = m_try

RETURN best_m_try
```

### Split Finding in Random Forests

**Standard Approach:**
```
ALGORITHM: RF Split Finding (Node-level)
INPUT: Node samples I, features F, m_try

1. feature_subset = RANDOM_SAMPLE(F, m_try)
2. best_gain = -∞
3. best_split = None

4. FOR each feature j in feature_subset:
   FOR each threshold θ in CANDIDATE_THRESHOLDS(j):
       gain = CALCULATE_IMPURITY_REDUCTION(j, θ, I)
       IF gain > best_gain:
           best_gain = gain
           best_split = (j, θ)

5. RETURN best_split
```

**Histogram-Based Optimization:**
For large datasets, use binning:

```
ALGORITHM: Histogram-Based RF Split
PARAMS: max_bins B (typically 32-255)

1. # Pre-compute histograms for sampled features
2. FOR each feature j in feature_subset:
   hist_j = BUILD_HISTOGRAM(samples[I, j], targets[I], B)
   
3. # Fast sweep over bins
4. FOR each feature j:
   FOR each bin_boundary θ in hist_j:
       gain = FAST_GAIN_FROM_HISTOGRAM(hist_j, θ)
       UPDATE_BEST_IF_BETTER(gain, (j, θ))
```

### Tree Growing Strategies

#### 1. Depth-First Growing (Standard)
```
ALGORITHM: Depth-First Tree Growing
FUNCTION GROW_NODE(samples, depth):
    IF STOPPING_CRITERION(samples, depth):
        RETURN LEAF(samples)
    
    split = FIND_BEST_SPLIT(samples)
    IF split is None:
        RETURN LEAF(samples)
    
    left_samples, right_samples = PARTITION(samples, split)
    left_child = GROW_NODE(left_samples, depth+1)
    right_child = GROW_NODE(right_samples, depth+1)
    
    RETURN INTERNAL_NODE(split, left_child, right_child)
```

#### 2. Breadth-First Growing
Better memory locality, easier parallelization:

```
ALGORITHM: Breadth-First Tree Growing
queue = [ROOT_NODE(all_samples)]

WHILE queue is not empty:
    current_level = queue
    next_level = []
    
    FOR node in current_level:
        IF not SHOULD_SPLIT(node):
            CONTINUE
        
        split = FIND_BEST_SPLIT(node.samples)
        left, right = PARTITION(node.samples, split)
        
        node.left = CREATE_NODE(left)
        node.right = CREATE_NODE(right)
        
        next_level.ADD(node.left, node.right)
    
    queue = next_level
```

### Stopping Criteria

**Multi-criteria Stopping:**
```
FUNCTION STOPPING_CRITERION(samples, depth):
    RETURN (
        LENGTH(samples) < min_samples_split OR
        depth >= max_depth OR
        PURITY(samples) > purity_threshold OR
        BEST_GAIN(samples) < min_impurity_decrease
    )
```

**Advanced Criteria:**

1. **Statistical Significance Testing:**
   ```
   chi2_stat = CALCULATE_CHI_SQUARE(left_dist, right_dist)
   p_value = CHI_SQUARE_TEST(chi2_stat, df=classes-1)
   IF p_value > significance_threshold:
       STOP_SPLITTING()
   ```

2. **Minimum Description Length (MDL):**
   ```
   current_cost = ENCODING_COST(samples)
   split_cost = ENCODING_COST(left) + ENCODING_COST(right) + SPLIT_OVERHEAD
   IF split_cost >= current_cost:
       STOP_SPLITTING()
   ```

---

## Extremely Randomized Trees

### Mathematical Motivation

**Bias-Variance Analysis:**
ExtraTrees introduces more bias but dramatically reduces variance:

- **Extra Bias:** From suboptimal splits
- **Variance Reduction:** From maximally decorrelated trees

**Expected Performance:**
$$\text{MSE}[\text{ExtraTrees}] = \text{Bias}^2[\text{RF}] + \Delta\text{Bias}^2 + \frac{\text{Var}[\text{RF}]}{C}$$

where $C > 1$ is the decorrelation factor and $\Delta\text{Bias}^2$ is additional bias from random splits.

### Split Generation Algorithms

#### 1. Uniform Random Splits
```
ALGORITHM: Uniform Random Split Generation
INPUT: Feature j, samples at node, K candidates

min_val = MIN(samples[:, j])
max_val = MAX(samples[:, j])

best_gain = -∞
best_threshold = None

FOR k = 1 to K:
    θ = UNIFORM_RANDOM(min_val, max_val)
    gain = CALCULATE_GAIN(j, θ, samples)
    IF gain > best_gain:
        best_gain = gain
        best_threshold = θ

RETURN (j, best_threshold, best_gain)
```

#### 2. Quantile-Based Random Splits
More principled than uniform sampling:

```
ALGORITHM: Quantile Random Split Generation
INPUT: Feature j, samples at node, K candidates

sorted_values = SORT(samples[:, j])
n_unique = LENGTH(UNIQUE(sorted_values))

# Generate K random quantiles
quantiles = UNIFORM_RANDOM(0, 1, K)
thresholds = [QUANTILE(sorted_values, q) for q in quantiles]

# Evaluate each threshold
FOR θ in thresholds:
    gain = CALCULATE_GAIN(j, θ, samples)
    UPDATE_BEST_IF_BETTER(gain, (j, θ))
```

#### 3. Histogram-Based Random Splits
For pre-binned features:

```
ALGORITHM: Histogram Random Split Generation
INPUT: Pre-computed histogram bins, K candidates

available_bins = NON_EMPTY_BINS(histogram)
selected_bins = RANDOM_SAMPLE(available_bins, min(K, len(available_bins)))

FOR bin_idx in selected_bins:
    θ = bin_boundaries[bin_idx]
    gain = FAST_GAIN_FROM_HISTOGRAM(bin_idx)
    UPDATE_BEST_IF_BETTER(gain, (feature_j, θ))
```

### ExtraTrees Hyperparameters

**Key Parameters:**
- **K (n_random_cuts):** Number of random thresholds per feature
  - Low K: More bias, less variance
  - High K: Approaches optimal splits
  - Typical: K = 1 (maximally random) to K = 10

- **Bootstrap:** Usually False (use full dataset)
  - Randomness comes from splits, not sampling
  - Can enable for very large datasets

- **Feature Selection:** Often use more features than RF
  - Since splits are suboptimal, need more features to compensate

**Hyperparameter Guidelines:**
```
ExtraTrees Configuration:
├─ K = 1: Maximum randomness, lowest correlation
├─ K = 3-5: Good balance for most problems  
├─ K = 10+: Approaches Random Forest behavior
└─ K = all: Equivalent to Random Forest

m_try (ExtraTrees):
├─ Often higher than RF: 2×√d to d
├─ Since splits are random, need more feature diversity
```

### Performance Characteristics

**Computational Complexity:**
- **Split Finding:** O(K × m_try) vs O(n × m_try) for RF
- **Memory:** Lower (no need to store/sort all split candidates)
- **Parallelization:** Excellent (less coordination needed)

**Statistical Properties:**
- **Consistency:** Still consistent under regularity conditions
- **Rate of Convergence:** Potentially slower than optimal methods
- **Robustness:** Very robust to outliers and noise

---

## Advanced Split Families

### Axis-Aligned Splits (Enhanced)

**Optimal Threshold Selection:**
For continuous features, consider multiple threshold selection strategies:

```
ALGORITHM: Multi-Strategy Threshold Selection
strategies = [
    "midpoint_between_classes",
    "entropy_weighted_mean", 
    "gini_optimal",
    "quantile_based"
]

FOR each strategy:
    thresholds = GENERATE_THRESHOLDS(feature, strategy)
    FOR θ in thresholds:
        gain = EVALUATE_SPLIT(feature, θ)
        UPDATE_GLOBAL_BEST()
```

**Missing Value Handling:**
```
ALGORITHM: Missing Value Split Evaluation  
# Three-way split: left, right, missing
FOR each threshold θ:
    left_samples = samples[feature <= θ]
    right_samples = samples[feature > θ]  
    missing_samples = samples[IS_MISSING(feature)]
    
    # Evaluate all possible assignments of missing values
    gain_left = IMPURITY_GAIN(left + missing, right, missing_penalty=0.1)
    gain_right = IMPURITY_GAIN(left, right + missing, missing_penalty=0.1)
    gain_separate = IMPURITY_GAIN(left, right, missing, missing_penalty=0.2)
    
    best_assignment = ARGMAX(gain_left, gain_right, gain_separate)
```

### Oblique Splits in Bagging

**Challenge:** Oblique splits can increase correlation between trees
**Solution:** Careful regularization and feature subsampling

```
ALGORITHM: Regularized Oblique Splits for Bagging
INPUT: Node samples, feature_subset, regularization λ

# Step 1: Feature selection for oblique split
IF LENGTH(feature_subset) > max_oblique_features:
    feature_subset = SELECT_TOP_K(feature_subset, max_oblique_features)

# Step 2: Fit regularized linear classifier
X_node = samples[:, feature_subset]
y_binary = BINARIZE_TARGETS(samples.targets)  # For split direction

# Ridge regression with strong regularization  
weights = RIDGE_REGRESSION(X_node, y_binary, alpha=λ)

# Step 3: Sparsify weights (feature selection)
weights = SOFT_THRESHOLD(weights, threshold=0.1 * MAX(ABS(weights)))

# Step 4: Find optimal threshold
projections = X_node @ weights
optimal_threshold = FIND_BEST_THRESHOLD(projections, y_binary)

RETURN ObliqueCondition(feature_subset, weights, optimal_threshold)
```

**Oblique Forest Hyperparameters:**
```
Oblique Random Forest:
├─ lambda: 0.1 - 1.0 (regularization strength)
├─ max_oblique_features: min(10, m_try) 
├─ sparsity_threshold: 0.05 - 0.2
└─ oblique_probability: 0.3 (fraction of oblique vs axial splits)
```

### Categorical Splits

**Multi-class Categorical Features:**
```
ALGORITHM: Optimal Categorical Split
INPUT: Categorical feature with levels {c_1, c_2, ..., c_K}

IF K <= 10:  # Small cardinality - exact search
    # Enumerate all 2^(K-1) possible binary partitions
    FOR each subset S ⊆ {c_1, ..., c_K}:
        gain = EVALUATE_BINARY_SPLIT(feature ∈ S vs feature ∉ S)
        UPDATE_BEST()

ELSE:  # Large cardinality - heuristic search
    # Order categories by target statistic
    stats = [MEAN_TARGET(samples[feature == c]) for c in categories]
    ordered_categories = SORT_BY(categories, stats)
    
    # Search contiguous prefixes (reduces search space)
    FOR i = 1 to K-1:
        S = ordered_categories[:i]
        gain = EVALUATE_BINARY_SPLIT(feature ∈ S vs feature ∉ S)
        UPDATE_BEST()
```

**High-Cardinality Categorical Features:**
```
ALGORITHM: Hash-Based Categorical Splits
INPUT: High-cardinality categorical feature

# Strategy 1: Feature hashing
hash_buckets = HASH_CATEGORIES_TO_BUCKETS(categories, n_buckets=32)
FOR each bucket_subset:
    EVALUATE_SPLIT(feature_hash ∈ bucket_subset)

# Strategy 2: Frequency-based grouping  
rare_categories = CATEGORIES_WITH_FREQUENCY_BELOW(threshold=0.01)
frequent_categories = COMPLEMENT(rare_categories)
splits = [
    frequent_categories vs rare_categories,
    TOP_K_FREQUENT vs REST,
    RANDOM_BINARY_PARTITION(frequent_categories)
]

# Strategy 3: Target-based clustering
category_embeddings = TARGET_STATISTIC_EMBEDDINGS(categories)
clusters = KMEANS(category_embeddings, k=4)  
FOR each cluster_subset:
    EVALUATE_SPLIT(feature ∈ cluster_subset)
```

---

## Out-of-Bag Estimation

### Mathematical Foundation

**OOB Error as Cross-Validation:**
For bootstrap sample $m$, let $\mathcal{O}_m$ be the OOB set. The OOB prediction for sample $i$ is:

$$\hat{y}_i^{\text{OOB}} = \frac{1}{|\{m : i \in \mathcal{O}_m\}|} \sum_{m: i \in \mathcal{O}_m} h^{(m)}(x_i)$$

**Theoretical Properties:**
- **Unbiased estimator** of generalization error under mild conditions
- **Variance:** Higher than k-fold CV due to uneven OOB set sizes
- **Computational cost:** Free (computed during training)

### Advanced OOB Techniques

#### 1. Weighted OOB Estimation
Account for varying OOB set sizes:

```
ALGORITHM: Weighted OOB Estimation
oob_predictions = {}
oob_weights = {}

FOR m = 1 to M:
    oob_indices = COMPLEMENT(bootstrap_indices[m])
    
    FOR i in oob_indices:
        IF i not in oob_predictions:
            oob_predictions[i] = 0
            oob_weights[i] = 0
            
        weight = 1.0 / SQRT(LENGTH(oob_indices))  # Weight by OOB set size
        oob_predictions[i] += weight * h_m(x_i)  
        oob_weights[i] += weight

# Normalize predictions
FOR i in oob_predictions:
    oob_predictions[i] /= oob_weights[i]

oob_error = CALCULATE_ERROR(y[oob_predictions.keys()], oob_predictions.values())
```

#### 2. Progressive OOB Monitoring
Track OOB error evolution during training:

```
ALGORITHM: Progressive OOB Monitoring
oob_scores = []
early_stopping_patience = 50
best_score = -∞
patience_counter = 0

FOR m = 1 to MAX_M:
    # Train next tree
    h_m = TRAIN_TREE(bootstrap_sample[m])
    ensemble.ADD(h_m)
    
    # Update OOB predictions
    UPDATE_OOB_PREDICTIONS(h_m, oob_indices[m])
    
    # Calculate current OOB score  
    current_oob = CALCULATE_OOB_ERROR()
    oob_scores.ADD(current_oob)
    
    # Early stopping logic
    IF current_oob > best_score:
        best_score = current_oob
        patience_counter = 0
    ELSE:
        patience_counter += 1
        IF patience_counter >= early_stopping_patience:
            BREAK
```

### OOB-Based Model Selection

**Hyperparameter Optimization:**
```
ALGORITHM: OOB Hyperparameter Selection
param_grid = {
    'max_depth': [5, 10, 15, None],
    'm_try': [√d, d/3, d/2, d], 
    'min_samples_leaf': [1, 5, 10, 20]
}

best_score = -∞
best_params = None

FOR param_combination in CARTESIAN_PRODUCT(param_grid):
    rf = RandomForest(**param_combination)
    rf.FIT(training_data)
    oob_score = rf.OOB_SCORE()
    
    IF oob_score > best_score:
        best_score = oob_score
        best_params = param_combination

final_model = RandomForest(**best_params)
```

**Feature Selection via OOB:**
```
ALGORITHM: OOB-Based Feature Selection  
current_features = ALL_FEATURES
best_oob_score = INITIAL_OOB_SCORE(current_features)

WHILE True:
    feature_importance = OOB_PERMUTATION_IMPORTANCE(current_features)
    
    # Remove least important feature
    least_important = ARGMIN(feature_importance)
    candidate_features = current_features - {least_important}
    
    candidate_oob_score = OOB_SCORE(candidate_features)
    
    IF candidate_oob_score >= best_oob_score - tolerance:
        current_features = candidate_features
        best_oob_score = candidate_oob_score
    ELSE:
        BREAK  # No improvement, stop selection

RETURN current_features
```

---

## Uncertainty Quantification

### Theoretical Framework

**Sources of Uncertainty in Bagging:**
1. **Epistemic (Model) Uncertainty:** Due to limited training data
2. **Aleatoric (Data) Uncertainty:** Inherent noise in observations
3. **Ensemble Disagreement:** Variation across ensemble members

### Bootstrap Confidence Intervals

#### 1. Percentile Method
For prediction $\hat{y}(x)$, collect bootstrap predictions:

$\{\hat{y}^{(1)}(x), \hat{y}^{(2)}(x), \ldots, \hat{y}^{(M)}(x)\}$

**Confidence Interval:**
$CI_{1-\alpha}(x) = [Q_{\alpha/2}(\{\hat{y}^{(m)}(x)\}), Q_{1-\alpha/2}(\{\hat{y}^{(m)}(x)\})]$

```
ALGORITHM: Bootstrap Percentile CI
INPUT: Test point x, ensemble {h_m}, confidence level α

predictions = [h_m(x) for m in 1..M]
lower_bound = QUANTILE(predictions, α/2)
upper_bound = QUANTILE(predictions, 1-α/2) 

RETURN [lower_bound, upper_bound]
```

#### 2. Bias-Corrected and Accelerated (BCa) Bootstrap
Adjusts for bias and skewness in the bootstrap distribution:

$CI_{BCa} = [Q_{\alpha_1}, Q_{\alpha_2}]$

where:
$\alpha_1 = \Phi\left(\hat{z}_0 + \frac{\hat{z}_0 + z_{\alpha/2}}{1 - \hat{a}(\hat{z}_0 + z_{\alpha/2})}\right)$

$\alpha_2 = \Phi\left(\hat{z}_0 + \frac{\hat{z}_0 + z_{1-\alpha/2}}{1 - \hat{a}(\hat{z}_0 + z_{1-\alpha/2})}\right)$

**Bias Correction Factor:**
$\hat{z}_0 = \Phi^{-1}\left(\frac{\#\{\hat{y}^{(m)}(x) < \hat{y}(x)\}}{M}\right)$

**Acceleration Parameter:**
$\hat{a} = \frac{\sum_{i}(\bar{u} - u_i)^3}{6[\sum_{i}(\bar{u} - u_i)^2]^{3/2}}$

where $u_i$ are jackknife estimates.

### Infinitesimal Jackknife (IJ)

**Theoretical Foundation:**
For each training sample $i$, compute its influence on the final prediction:

$\text{IJ}_i(x) = \frac{\partial \hat{y}(x)}{\partial w_i}$

where $w_i$ is the weight of sample $i$.

**Variance Estimation:**
$\hat{\text{Var}}[\hat{y}(x)] = \sum_{i=1}^n \text{IJ}_i(x)^2$

```
ALGORITHM: Infinitesimal Jackknife for Random Forest
INPUT: Random Forest ensemble, test point x

total_variance = 0

FOR each training sample i:
    influence_i = 0
    
    FOR each tree m:
        IF i was in bootstrap sample m:
            # Compute influence of sample i on tree m's prediction
            influence_m = COMPUTE_TREE_INFLUENCE(tree_m, sample_i, x)
            influence_i += influence_m / M
    
    total_variance += influence_i^2

prediction_std = SQRT(total_variance)
RETURN prediction_std
```

### Quantile Regression Forests

**Core Principle:** Instead of predicting point estimates, maintain full conditional distributions at leaves.

**Algorithm:**
```
ALGORITHM: Quantile Regression Forest Training
INPUT: Training data D, quantiles Q = {q_1, q_2, ..., q_k}

FOR each tree m:
    D_m = BOOTSTRAP_SAMPLE(D)
    tree_m = BUILD_TREE(D_m)
    
    FOR each leaf L in tree_m:
        # Store all target values that reach this leaf
        leaf_targets[L] = [y_i for (x_i, y_i) in D_m if REACHES_LEAF(x_i, L)]

FUNCTION PREDICT_QUANTILES(x, quantiles):
    all_targets = []
    
    FOR each tree m:
        leaf = FIND_LEAF(tree_m, x)
        all_targets.EXTEND(leaf_targets[leaf])
    
    # Return empirical quantiles
    RETURN [QUANTILE(all_targets, q) for q in quantiles]
```

**Prediction Intervals:**
For confidence level $1-\alpha$:

$PI_{1-\alpha}(x) = [Q_{\alpha/2}(Y|X=x), Q_{1-\alpha/2}(Y|X=x)]$

### Conformal Prediction with Random Forests

**Split Conformal Framework:**
1. Split data: $\mathcal{D} = \mathcal{D}_{\text{train}} \cup \mathcal{D}_{\text{cal}}$
2. Train ensemble on $\mathcal{D}_{\text{train}}$
3. Compute nonconformity scores on $\mathcal{D}_{\text{cal}}$
4. Use quantiles of scores for prediction intervals

```
ALGORITHM: Conformal Prediction Intervals
INPUT: Trained RF, calibration set D_cal, confidence α

# Step 1: Compute nonconformity scores
nonconformity_scores = []
FOR (x_i, y_i) in D_cal:
    pred_i = RF.PREDICT(x_i)
    score_i = ABS(y_i - pred_i)  # Or other score function
    nonconformity_scores.ADD(score_i)

# Step 2: Find quantile threshold
n_cal = LENGTH(D_cal)
q_level = (n_cal + 1) * (1 - α) / n_cal
threshold = QUANTILE(nonconformity_scores, q_level)

# Step 3: Prediction intervals for new points
FUNCTION PREDICT_INTERVAL(x_new):
    pred = RF.PREDICT(x_new)
    RETURN [pred - threshold, pred + threshold]
```

**Adaptive Conformal Prediction:**
Adjust interval width based on local prediction difficulty:

```
ALGORITHM: Locally Adaptive Conformal Prediction
# Use ensemble disagreement as difficulty measure
FOR (x_i, y_i) in D_cal:
    individual_preds = [tree.PREDICT(x_i) for tree in RF.trees]
    disagreement_i = STANDARD_DEVIATION(individual_preds)
    
    # Normalized nonconformity score
    score_i = ABS(y_i - MEAN(individual_preds)) / (disagreement_i + ε)
    adaptive_scores.ADD(score_i)

# Use adaptive scores for threshold
adaptive_threshold = QUANTILE(adaptive_scores, q_level)

FUNCTION ADAPTIVE_PREDICT_INTERVAL(x_new):
    individual_preds = [tree.PREDICT(x_new) for tree in RF.trees]
    pred_mean = MEAN(individual_preds)
    disagreement = STANDARD_DEVIATION(individual_preds)
    
    interval_width = adaptive_threshold * (disagreement + ε)
    RETURN [pred_mean - interval_width, pred_mean + interval_width]
```

---

## Feature Importance Methods

### Split-Based Importance (Mean Decrease Impurity)

**Mathematical Definition:**
For feature $j$, the importance is:

$I_j = \frac{1}{M} \sum_{m=1}^M \sum_{t \in T_m: v(t)=j} p(t) \cdot \Delta I(t)$

where:
- $v(t)$ is the feature used at node $t$
- $p(t) = \frac{N_t}{N}$ is the fraction of samples reaching node $t$
- $\Delta I(t)$ is the impurity decrease from splitting node $t$

```
ALGORITHM: MDI Feature Importance
INPUT: Random Forest ensemble

importance = ZEROS(n_features)

FOR each tree T in ensemble:
    FOR each internal node t in T:
        feature_j = t.split_feature
        n_samples_t = t.n_samples
        impurity_decrease = t.impurity_decrease
        
        # Weight by fraction of samples
        weight = n_samples_t / total_training_samples
        importance[feature_j] += weight * impurity_decrease

# Normalize to sum to 1
importance = importance / SUM(importance)
RETURN importance
```

**Limitations:**
- Biased toward high-cardinality categorical features
- Doesn't account for feature interactions
- Can assign importance to irrelevant correlated features

### Permutation Importance

**Core Principle:** Measure performance degradation when feature values are randomly shuffled.

```
ALGORITHM: Permutation Importance (OOB)
INPUT: Trained Random Forest, OOB samples, metric, n_repeats

baseline_score = OOB_SCORE(RF)
importances = ZEROS(n_features)

FOR feature_j in 1..n_features:
    scores = []
    
    FOR repeat in 1..n_repeats:
        # Create permuted OOB dataset
        X_oob_permuted = COPY(X_oob)
        X_oob_permuted[:, feature_j] = SHUFFLE(X_oob[:, feature_j])
        
        # Compute score with permuted feature
        permuted_score = SCORE(RF, X_oob_permuted, y_oob, metric)
        scores.ADD(permuted_score)
    
    # Importance = average performance drop
    importances[feature_j] = baseline_score - MEAN(scores)

RETURN importances
```

**Advanced Permutation Strategies:**

#### 1. Conditional Permutation
Preserves feature correlations:

```
ALGORITHM: Conditional Permutation Importance
INPUT: Feature j to permute, conditioning features C

FOR each unique combination of values in features C:
    subset_indices = INDICES_WHERE(X[:, C] == combination)
    
    # Permute only within this subset
    X_permuted[subset_indices, j] = SHUFFLE(X[subset_indices, j])
```

#### 2. Group Permutation
For correlated feature groups:

```
ALGORITHM: Group Permutation Importance
INPUT: Feature groups G = {G_1, G_2, ..., G_k}

FOR each group G_i:
    # Permute all features in group together
    permutation_order = SHUFFLE(RANGE(n_samples))
    FOR feature_j in G_i:
        X_permuted[:, feature_j] = X[permutation_order, feature_j]
    
    group_importance[G_i] = baseline_score - SCORE(X_permuted, y)
```

### SHAP Values for Tree Ensembles

**TreeSHAP Algorithm:**
Efficiently computes SHAP values for tree ensembles using:

$\phi_j = \sum_{S \subseteq \mathcal{F} \setminus \{j\}} \frac{|S|!(|\mathcal{F}|-|S|-1)!}{|\mathcal{F}|!} [f_x(S \cup \{j\}) - f_x(S)]$

```
ALGORITHM: TreeSHAP for Random Forest
INPUT: Tree ensemble, instance x, background dataset

shap_values = ZEROS(n_features)

FOR each tree T in ensemble:
    tree_shap = COMPUTE_TREE_SHAP(T, x, background)
    shap_values += tree_shap / n_trees

FUNCTION COMPUTE_TREE_SHAP(tree, x, background):
    # Recursive computation using tree structure
    RETURN RECURSIVE_SHAP(tree.root, x, background, path=[], 
                         zero_fraction=1.0, one_fraction=1.0)
```

**SHAP Interaction Values:**
Capture pairwise feature interactions:

$\phi_{i,j} = \sum_{S \subseteq \mathcal{F} \setminus \{i,j\}} \frac{|S|!(|\mathcal{F}|-|S|-2)!}{2(|\mathcal{F}|-1)!} \Delta_{ij}(S)$

where $\Delta_{ij}(S) = f(S \cup \{i,j\}) - f(S \cup \{i\}) - f(S \cup \{j\}) + f(S)$

### Feature Selection Based on Importance

**Recursive Feature Elimination with Cross-Validation:**
```
ALGORITHM: RFE-CV with Random Forest
INPUT: Features F, target y, cv_folds, min_features

feature_ranking = []
current_features = F

WHILE LENGTH(current_features) > min_features:
    # Train RF and compute importance
    rf = TRAIN_RF(current_features, y)
    importance = COMPUTE_IMPORTANCE(rf)
    
    # Cross-validation score with current features
    cv_score = CROSS_VAL_SCORE(rf, current_features, y, cv_folds)
    feature_ranking.ADD((current_features, cv_score))
    
    # Remove least important feature
    least_important = ARGMIN(importance)
    current_features.REMOVE(least_important)

# Select feature set with best CV score
best_features = ARGMAX(feature_ranking, key=lambda x: x[1])
RETURN best_features
```

**Stability-Based Selection:**
```
ALGORITHM: Stability-Based Feature Selection
INPUT: Dataset D, bootstrap_samples B, stability_threshold

stability_scores = ZEROS(n_features)

FOR b in 1..B:
    D_boot = BOOTSTRAP_SAMPLE(D)
    rf_boot = TRAIN_RF(D_boot)
    importance_boot = COMPUTE_IMPORTANCE(rf_boot)
    
    # Binarize importance (top-k selection)
    top_k_features = TOP_K_INDICES(importance_boot, k)
    binary_selection = ZEROS(n_features)
    binary_selection[top_k_features] = 1
    
    stability_scores += binary_selection

# Stability = frequency of selection across bootstraps
stability_scores /= B
stable_features = INDICES_WHERE(stability_scores >= stability_threshold)

RETURN stable_features
```

---

## Handling Special Cases

### Imbalanced Classification

#### 1. Balanced Random Forest
```
ALGORITHM: Balanced Random Forest
INPUT: Imbalanced dataset D, minority_class_ratio

FOR each tree m:
    # Balanced bootstrap sampling
    D_m = BALANCED_BOOTSTRAP_SAMPLE(D, minority_class_ratio)
    tree_m = TRAIN_TREE(D_m)
    ensemble.ADD(tree_m)

FUNCTION BALANCED_BOOTSTRAP_SAMPLE(D, target_ratio):
    minority_samples = D[y == minority_class]
    majority_samples = D[y == majority_class]
    
    n_minority = LENGTH(minority_samples)
    n_majority_target = n_minority / target_ratio
    
    # Oversample minority, undersample majority
    D_minority_boot = BOOTSTRAP_SAMPLE(minority_samples, n_minority)
    D_majority_boot = SAMPLE_WITHOUT_REPLACEMENT(majority_samples, n_majority_target)
    
    RETURN CONCATENATE(D_minority_boot, D_majority_boot)
```

#### 2. Cost-Sensitive Splitting
Modify impurity measures to account for misclassification costs:

```
ALGORITHM: Cost-Sensitive Gini Impurity
INPUT: Node samples, cost matrix C

weighted_gini = 0
total_cost_weighted_samples = 0

FOR each class i:
    n_i = COUNT(samples, class=i)
    cost_weight_i = SUM(C[i, :])  # Cost of misclassifying class i
    
    weighted_gini += cost_weight_i * n_i * (N - n_i) / N
    total_cost_weighted_samples += cost_weight_i * n_i

RETURN weighted_gini / total_cost_weighted_samples
```

#### 3. Threshold Optimization
Use OOB predictions to find optimal classification threshold:

```
ALGORITHM: OOB Threshold Optimization
INPUT: RF with OOB predictions, optimization metric

oob_probabilities = RF.OOB_PREDICT_PROBA()
true_labels = y[oob_indices]

# Grid search over thresholds
thresholds = LINSPACE(0.1, 0.9, 50)
best_threshold = 0.5
best_score = -inf

FOR threshold in thresholds:
    predictions = oob_probabilities[:, 1] >= threshold
    score = CALCULATE_METRIC(true_labels, predictions)
    
    IF score > best_score:
        best_score = score
        best_threshold = threshold

RF.decision_threshold = best_threshold
```

### Multi-Output Problems

#### 1. Multi-Output Regression
```
ALGORITHM: Multi-Output Random Forest
INPUT: X, Y where Y has k targets

FOR each tree m:
    D_m = BOOTSTRAP_SAMPLE(X, Y)
    tree_m = BUILD_MULTI_OUTPUT_TREE(D_m)

FUNCTION BUILD_MULTI_OUTPUT_TREE(D):
    # Use vector impurity measures
    FUNCTION VECTOR_MSE_IMPURITY(node):
        predictions = MEAN(node.targets, axis=0)  # k-dimensional mean
        mse = MEAN(SUM((node.targets - predictions)^2, axis=1))
        RETURN mse
    
    # Split selection remains the same
    best_split = FIND_BEST_SPLIT(node, VECTOR_MSE_IMPURITY)
```

#### 2. Multi-Label Classification
```
ALGORITHM: Multi-Label Random Forest
approaches = [
    "binary_relevance",    # Independent RF for each label
    "label_powerset",      # Treat unique label combinations as classes  
    "classifier_chains"    # Chain predictions (label dependencies)
]

FUNCTION BINARY_RELEVANCE(X, Y):
    forests = {}
    FOR each label l:
        forests[l] = TRAIN_RF(X, Y[:, l])
    
    FUNCTION PREDICT(x):
        predictions = {}
        FOR each label l:
            predictions[l] = forests[l].PREDICT_PROBA(x)[1]
        RETURN predictions

FUNCTION CLASSIFIER_CHAINS(X, Y):
    label_order = RANDOM_PERMUTATION(n_labels)
    chain_forests = {}
    
    FOR i, label_l in ENUMERATE(label_order):
        # Features = original + predictions of previous labels in chain
        if i == 0:
            X_augmented = X
        else:
            prev_predictions = [chain_forests[prev_l].PREDICT(X) 
                              for prev_l in label_order[:i]]
            X_augmented = CONCATENATE(X, prev_predictions)
        
        chain_forests[label_l] = TRAIN_RF(X_augmented, Y[:, label_l])
```

### Missing Value Handling

#### 1. Surrogate Splits
When primary split feature is missing, use correlated features:

```
ALGORITHM: Surrogate Split Selection
INPUT: Primary split (feature j, threshold θ), node samples

# Find best surrogate splits
surrogates = []
primary_direction = [LEFT if x[i,j] <= θ else RIGHT for i in node_samples]

FOR each other feature k != j:
    FOR each threshold φ:
        surrogate_direction = [LEFT if x[i,k] <= φ else RIGHT for i in node_samples]
        agreement = PROPORTION_AGREEMENT(primary_direction, surrogate_direction)
        
        IF agreement > min_surrogate_agreement:
            surrogates.ADD((feature=k, threshold=φ, agreement=agreement))

# Sort by agreement, keep top surrogates
surrogates = SORT_BY(surrogates, agreement, descending=True)
RETURN surrogates[:max_surrogates]
```

#### 2. Probabilistic Routing
Route missing values probabilistically based on training data:

```
ALGORITHM: Probabilistic Missing Value Routing
INPUT: Node with missing values for split feature

training_samples_at_node = node.training_samples
left_probability = COUNT(training_samples_at_node, went_left) / COUNT(training_samples_at_node)
right_probability = 1 - left_probability

FUNCTION PREDICT_WITH_MISSING(x):
    IF x[split_feature] is not MISSING:
        RETURN STANDARD_ROUTING(x)
    ELSE:
        # Probabilistic prediction
        left_pred = left_probability * LEFT_SUBTREE.PREDICT(x)
        right_pred = right_probability * RIGHT_SUBTREE.PREDICT(x)
        RETURN left_pred + right_pred
```

### High-Dimensional Data

#### 1. Feature Hashing
For extremely high-dimensional sparse features:

```
ALGORITHM: Feature Hashing for RF
INPUT: Sparse feature matrix X, hash_size H

X_hashed = ZEROS(n_samples, H)

FOR sample i:
    FOR each nonzero feature (j, value):
        hash_index = HASH(j) % H
        X_hashed[i, hash_index] += value  # Handle collisions by addition

rf = TRAIN_RF(X_hashed, y)
```

#### 2. Random Projections
Dimensionality reduction while preserving distances:

```
ALGORITHM: Random Projection RF
INPUT: High-dimensional X, target dimensionality d_target

# Generate random projection matrix
R = GAUSSIAN_RANDOM_MATRIX(d_original, d_target)
R = R / NORM(R, axis=0)  # Normalize columns

X_projected = X @ R
rf = TRAIN_RF(X_projected, y)

FUNCTION PREDICT(x_new):
    x_projected = x_new @ R
    RETURN rf.PREDICT(x_projected)
```

---

## Performance Optimization

### Memory Optimization

#### 1. Compact Tree Representation
```
ALGORITHM: Memory-Efficient Tree Storage
# Instead of storing full tree objects, use arrays

# Tree structure arrays (for tree t)
left_children[t] = array of left child indices
right_children[t] = array of right child indices  
split_features[t] = array of split feature indices
split_thresholds[t] = array of split threshold values
leaf_values[t] = array of leaf prediction values
is_leaf[t] = boolean array indicating leaf nodes

FUNCTION PREDICT_COMPACT(x, tree_index):
    node = 0  # Start at root
    
    WHILE not is_leaf[tree_index][node]:
        feature = split_features[tree_index][node]
        threshold = split_thresholds[tree_index][node]
        
        IF x[feature] <= threshold:
            node = left_children[tree_index][node]
        ELSE:
            node = right_children[tree_index][node]
    
    RETURN leaf_values[tree_index][node]
```

#### 2. Feature Binning and Quantization
```
ALGORITHM: Memory-Efficient Feature Storage
# Pre-bin continuous features
FOR each feature j:
    IF IS_CONTINUOUS(feature_j):
        bins_j = QUANTILE_BINS(X[:, j], n_bins=255)
        X_binned[:, j] = DIGITIZE(X[:, j], bins_j)
        
        # Store bins with 8-bit integers instead of 64-bit floats
        X_quantized[:, j] = X_binned[:, j].astype(uint8)
```

### Computational Optimization

#### 1. Vectorized Prediction
```
ALGORITHM: Batch Prediction Optimization
INPUT: Test batch X_test, ensemble

# Vectorized tree traversal
predictions = ZEROS(n_test_samples, n_trees)

FOR tree_idx in 1..n_trees:
    node_indices = ZEROS(n_test_samples, dtype=int)  # Start at root (0) for all
    
    # Traverse all samples simultaneously
    WHILE any(not is_leaf[tree_idx][node_indices]):
        active_mask = not is_leaf[tree_idx][node_indices]
        
        features = split_features[tree_idx][node_indices[active_mask]]
        thresholds = split_thresholds[tree_idx][node_indices[active_mask]]
        
        # Vectorized comparison
        go_left = X_test[active_mask, features] <= thresholds
        
        # Update node indices
        node_indices[active_mask & go_left] = left_children[tree_idx][node_indices[active_mask & go_left]]
        node_indices[active_mask & ~go_left] = right_children[tree_idx][node_indices[active_mask & ~go_left]]
    
    # Collect leaf predictions
    predictions[:, tree_idx] = leaf_values[tree_idx][node_indices]

# Final ensemble prediction
RETURN MEAN(predictions, axis=1)
```

#### 2. Parallel Tree Construction
```
ALGORITHM: Parallel Random Forest Training
INPUT: Data D, n_trees M, n_threads T

# Divide trees across threads
trees_per_thread = M // T
thread_assignments = [RANGE(i*trees_per_thread, (i+1)*trees_per_thread) 
                     for i in RANGE(T)]

PARALLEL FOR thread_id in 1..T:
    local_trees = []
    
    FOR tree_idx in thread_assignments[thread_id]:
        bootstrap_sample = BOOTSTRAP_SAMPLE(D)
        tree = BUILD_TREE(bootstrap_sample)
        local_trees.ADD(tree)
    
    thread_results[thread_id] = local_trees

# Combine results
ensemble = CONCATENATE([thread_results[i] for i in 1..T])
```

#### 3. NUMA-Aware Processing
```
ALGORITHM: NUMA-Aware Random Forest
# Bind threads to NUMA nodes for better memory locality

FUNCTION NUMA_AWARE_TRAINING():
    numa_nodes = GET_NUMA_TOPOLOGY()
    
    FOR numa_node in numa_nodes:
        # Bind threads to this NUMA node
        SET_THREAD_AFFINITY(numa_node.cpu_cores)
        
        # Allocate data on local memory
        local_data = COPY_TO_NUMA_NODE(training_data, numa_node)
        
        # Train subset of trees
        trees_for_node = n_trees // n_numa_nodes
        local_ensemble = TRAIN_TREES(local_data, trees_for_node)
        
        global_ensemble.EXTEND(local_ensemble)
```

### Scalability Optimizations

#### 1. Online/Streaming Random Forest
```
ALGORITHM: Online Random Forest
STATE: Ensemble of online trees {T_1, T_2, ..., T_M}

FUNCTION UPDATE(new_sample):
    FOR each tree T_m:
        # Each tree sees sample with Poisson(λ=1) frequency
        k = POISSON_RANDOM(1)
        FOR _ in 1..k:
            T_m.UPDATE(new_sample)

FUNCTION ONLINE_TREE_UPDATE(tree, sample):
    # Traverse to leaf
    leaf = FIND_LEAF(tree, sample.x)
    
    # Update leaf statistics
    leaf.sum_targets += sample.y
    leaf.sum_squared_targets += sample.y^2
    leaf.n_samples += 1
    
    # Potentially split leaf if it has grown large enough
    IF leaf.n_samples > min_samples_to_split:
        ATTEMPT_SPLIT(leaf)
```

#### 2. Distributed Random Forest
```
ALGORITHM: Map-Reduce Random Forest
FUNCTION MAP_PHASE(data_partition):
    # Each worker trains subset of trees
    local_trees = []
    FOR i in 1..trees_per_worker:
        bootstrap_sample = BOOTSTRAP_SAMPLE(data_partition)
        tree = BUILD_TREE(bootstrap_sample)
        local_trees.ADD(tree)
    
    RETURN local_trees

FUNCTION REDUCE_PHASE(all_tree_collections):
    # Combine trees from all workers
    global_ensemble = []
    FOR tree_collection in all_tree_collections:
        global_ensemble.EXTEND(tree_collection)
    
    RETURN RandomForestEnsemble(global_ensemble)

# Usage
partitioned_data = PARTITION_DATA(training_data, n_workers)
tree_collections = PARALLEL_MAP(MAP_PHASE, partitioned_data)
final_model = REDUCE_PHASE(tree_collections)
```

---

## Advanced Topics

### Mondrian Forests

**Time-Structured Splits:** Instead of axis-aligned splits, use "Mondrian" cuts that respect temporal structure.

```
ALGORITHM: Mondrian Forest
INPUT: Spatio-temporal data with timestamps

FUNCTION MONDRIAN_SPLIT(node_data):
    # Sample split time exponentially
    τ = node.creation_time + EXPONENTIAL_RANDOM(rate=1)
    
    # Choose dimension proportional to box extent
    extents = [MAX(node_data[:, j]) - MIN(node_data[:, j]) for j in features]
    total_extent = SUM(extents)
    split_dim = CATEGORICAL_SAMPLE(features, weights=extents/total_extent)
    
    # Uniform split location within dimension
    min_val, max_val = MIN_MAX(node_data[:, split_dim])
    split_location = UNIFORM(min_val, max_val)
    
    RETURN Mondriansplit(time=τ, dimension=split_dim, location=split_location)
```

**Applications:**
- Online learning with temporal consistency
- Spatial data with geographical structure
- Streaming data with concept drift

### Isolation Forests

**Anomaly Detection via Random Partitioning:**

```
ALGORITHM: Isolation Forest
INPUT: Dataset X (no labels), contamination_rate

FOR tree in 1..n_trees:
    # Build isolation tree
    tree = BUILD_ISOLATION_TREE(X, max_depth=ceil(log2(n_samples)))

FUNCTION BUILD_ISOLATION_TREE(data, current_depth):
    IF current_depth >= max_depth OR |data| <= 1:
        RETURN LEAF(size=|data|)
    
    # Random split
    split_feature = RANDOM_CHOICE(features)
    min_val, max_val = MIN_MAX(data[:, split_feature])
    split_point = UNIFORM(min_val, max_val)
    
    left_data = data[data[:, split_feature] < split_point]
    right_data = data[data[:, split_feature] >= split_point]
    
    left_child = BUILD_ISOLATION_TREE(left_data, current_depth + 1)
    right_child = BUILD_ISOLATION_TREE(right_data, current_depth + 1)
    
    RETURN INTERNAL_NODE(split_feature, split_point, left_child, right_child)

FUNCTION ANOMALY_SCORE(x):
    path_lengths = []
    FOR tree in isolation_forest:
        path_length = COMPUTE_PATH_LENGTH(tree, x)
        path_lengths.ADD(path_length)
    
    average_path_length = MEAN(path_lengths)
    # Normalize by expected path length in BST
    c_n = 2 * (ln(n_samples - 1) + 0.5772156649) - (2 * (n_samples - 1) / n_samples)
    anomaly_score = 2^(-average_path_length / c_n)
    
    RETURN anomaly_score
```

**Anomaly Threshold Selection:**
```
# Anomaly scores close to 1 indicate anomalies
# Scores close to 0 indicate normal points
contamination_threshold = QUANTILE(all_scores, 1 - contamination_rate)
anomalies = INDICES_WHERE(scores > contamination_threshold)
```

### Random Survival Forests

**Extension to Censored Data:**

```
ALGORITHM: Random Survival Forest
INPUT: Training data (X, T, δ) where T=event times, δ=censoring indicators

FOR each tree m:
    bootstrap_sample = BOOTSTRAP_SAMPLE(X, T, δ)
    survival_tree = BUILD_SURVIVAL_TREE(bootstrap_sample)

FUNCTION BUILD_SURVIVAL_TREE(data):
    IF STOPPING_CRITERION(data):
        # Leaf contains Kaplan-Meier estimator
        km_estimator = KAPLAN_MEIER(data.T, data.δ)
        RETURN SURVIVAL_LEAF(km_estimator)
    
    best_split = FIND_BEST_SURVIVAL_SPLIT(data)
    left_data, right_data = PARTITION(data, best_split)
    
    left_child = BUILD_SURVIVAL_TREE(left_data)
    right_child = BUILD_SURVIVAL_TREE(right_data)
    
    RETURN SURVIVAL_NODE(best_split, left_child, right_child)

FUNCTION FIND_BEST_SURVIVAL_SPLIT(data):
    # Use log-rank test statistic as splitting criterion
    best_score = 0
    best_split = None
    
    FOR feature_j in RANDOM_SAMPLE(features, m_try):
        FOR threshold θ:
            left_indices = data.X[:, feature_j] <= θ
            right_indices = ~left_indices
            
            # Log-rank test between left and right groups
            log_rank_score = LOG_RANK_TEST(
                data.T[left_indices], data.δ[left_indices],
                data.T[right_indices], data.δ[right_indices]
            )
            
            IF log_rank_score > best_score:
                best_score = log_rank_score
                best_split = (feature_j, θ)
    
    RETURN best_split

FUNCTION PREDICT_SURVIVAL(x, times):
    survival_functions = []
    
    FOR tree in forest:
        leaf = FIND_LEAF(tree, x)
        survival_func = leaf.kaplan_meier_estimator
        survival_functions.ADD(survival_func)
    
    # Average survival functions
    ensemble_survival = MEAN([sf.evaluate(times) for sf in survival_functions])
    RETURN ensemble_survival
```

### Causal Forests

**Heterogeneous Treatment Effect Estimation:**

```
ALGORITHM: Causal Forest (Generalized Random Forest)
INPUT: Features X, treatment W, outcomes Y

# Honest splitting: different samples for splitting and estimation
FOR each tree m:
    bootstrap_sample = BOOTSTRAP_SAMPLE(X, W, Y)
    splitting_sample, estimation_sample = SPLIT_SAMPLE(bootstrap_sample, ratio=0.5)
    
    causal_tree = BUILD_CAUSAL_TREE(splitting_sample, estimation_sample)

FUNCTION BUILD_CAUSAL_TREE(splitting_data, estimation_data):
    IF STOPPING_CRITERION(splitting_data):
        # Estimate treatment effect in leaf using estimation sample
        leaf_indices = FIND_ESTIMATION_INDICES(estimation_data, current_region)
        treatment_effect = ESTIMATE_CATE(estimation_data[leaf_indices])
        RETURN CAUSAL_LEAF(treatment_effect)
    
    # Find split that maximizes treatment effect heterogeneity
    best_split = FIND_BEST_CAUSAL_SPLIT(splitting_data)
    
    left_splitting, right_splitting = PARTITION(splitting_data, best_split)
    left_estimation, right_estimation = PARTITION(estimation_data, best_split)
    
    left_child = BUILD_CAUSAL_TREE(left_splitting, left_estimation)
    right_child = BUILD_CAUSAL_TREE(right_splitting, right_estimation)
    
    RETURN CAUSAL_NODE(best_split, left_child, right_child)

FUNCTION FIND_BEST_CAUSAL_SPLIT(data):
    # Splitting criterion: maximize variance of treatment effects across children
    best_criterion = 0
    best_split = None
    
    FOR feature_j in RANDOM_SAMPLE(features, m_try):
        FOR threshold θ:
            left_data = data[data.X[:, feature_j] <= θ]
            right_data = data[data.X[:, feature_j] > θ]
            
            # Estimate treatment effects in each child
            tau_left = ESTIMATE_CATE(left_data)
            tau_right = ESTIMATE_CATE(right_data)
            
            # Criterion: weighted variance of treatment effects
            n_left, n_right = |left_data|, |right_data|
            n_total = n_left + n_right
            
            criterion = (n_left * n_right / n_total) * (tau_left - tau_right)^2
            
            IF criterion > best_criterion:
                best_criterion = criterion
                best_split = (feature_j, θ)
    
    RETURN best_split

FUNCTION ESTIMATE_CATE(data):
    # Simple difference-in-means estimator
    treated_outcomes = MEAN(data.Y[data.W == 1])
    control_outcomes = MEAN(data.Y[data.W == 0])
    RETURN treated_outcomes - control_outcomes
```

### Multi-Armed Bandit Forests

**Contextual Bandits with Thompson Sampling:**

```
ALGORITHM: Contextual Bandit Forest
STATE: Forest of contextual trees, reward histories

FUNCTION SELECT_ACTION(context_x):
    # Thompson sampling: sample from posterior distribution
    sampled_rewards = []
    
    FOR each action a:
        # Get leaf predictions for this context-action pair
        leaf_rewards = []
        FOR tree in forest:
            leaf = FIND_LEAF(tree, CONCATENATE(context_x, [a]))
            # Sample from leaf's reward distribution
            posterior_sample = BETA_SAMPLE(leaf.alpha, leaf.beta)  # For binary rewards
            leaf_rewards.ADD(posterior_sample)
        
        expected_reward = MEAN(leaf_rewards)
        sampled_rewards.ADD(expected_reward)
    
    # Select action with highest sampled reward
    best_action = ARGMAX(sampled_rewards)
    RETURN best_action

FUNCTION UPDATE_FOREST(context_x, action_a, reward_r):
    context_action = CONCATENATE(context_x, [action_a])
    
    FOR tree in forest:
        leaf = FIND_LEAF(tree, context_action)
        # Update Beta distribution parameters
        IF reward_r == 1:
            leaf.alpha += 1
        ELSE:
            leaf.beta += 1
        
        # Optionally rebuild tree if leaf has enough data
        IF leaf.total_updates % rebuild_threshold == 0:
            REBUILD_TREE_BRANCH(tree, leaf)
```

---

## Implementation Guidelines

### Architecture Design

#### 1. Core Interfaces

```
INTERFACE: BaseTree
METHODS:
    fit(X, y, sample_weight=None)
    predict(X)
    get_leaf_indices(X)
    get_n_leaves()
    get_depth()

INTERFACE: TreeBuilder  
METHODS:
    build_tree(X, y, sample_weight=None)
    find_best_split(node_samples, feature_indices)
    create_leaf(node_samples)
    should_split(node_samples)

INTERFACE: Splitter
METHODS:
    find_split(X, y, feature_indices, sample_indices)
    evaluate_split(feature, threshold, sample_indices)

INTERFACE: EnsembleMethod
METHODS:
    fit(X, y)
    predict(X) 
    predict_proba(X)
    get_feature_importance()
    get_oob_score()
```

#### 2. Modular Design Pattern

```
CLASS RandomForest:
    CONSTRUCTOR(
        tree_builder: TreeBuilder,
        splitter: Splitter, 
        bootstrap_sampler: BootstrapSampler,
        feature_sampler: FeatureSampler
    )
    
    FUNCTION fit(X, y):
        self.trees = []
        self.oob_predictions = ZEROS(len(y))
        self.oob_counts = ZEROS(len(y))
        
        FOR tree_idx in range(self.n_estimators):
            # Get bootstrap sample
            sample_indices = self.bootstrap_sampler.sample(len(X))
            oob_indices = COMPLEMENT(sample_indices, len(X))
            
            # Train tree
            X_bootstrap = X[sample_indices]
            y_bootstrap = y[sample_indices]
            
            tree = self.tree_builder.build_tree(X_bootstrap, y_bootstrap)
            self.trees.append(tree)
            
            # Update OOB predictions
            IF len(oob_indices) > 0:
                oob_preds = tree.predict(X[oob_indices])
                self.oob_predictions[oob_indices] += oob_preds
                self.oob_counts[oob_indices] += 1
        
        # Finalize OOB predictions
        mask = self.oob_counts > 0
        self.oob_predictions[mask] /= self.oob_counts[mask]
```

### Data Structures

#### 1. Efficient Tree Representation

```
STRUCTURE: CompactTree
FIELDS:
    nodes: Array[TreeNode]           # All nodes in breadth-first order
    feature_indices: Array[int16]    # Split features (16-bit for memory)
    thresholds: Array[float32]       # Split thresholds (32-bit precision)
    left_children: Array[int32]      # Left child indices
    right_children: Array[int32]     # Right child indices  
    leaf_values: Array[float32]      # Leaf predictions
    is_leaf: Array[bool]            # Leaf indicators

FUNCTION traverse(x: sample) -> prediction:
    node_idx = 0  # Start at root
    
    WHILE not self.is_leaf[node_idx]:
        feature_idx = self.feature_indices[node_idx]
        threshold = self.thresholds[node_idx]
        
        IF x[feature_idx] <= threshold:
            node_idx = self.left_children[node_idx]
        ELSE:
            node_idx = self.right_children[node_idx]
    
    RETURN self.leaf_values[node_idx]
```

#### 2. Memory-Mapped Data Loading

```
STRUCTURE: MemoryMappedDataset
FIELDS:
    X_memmap: np.memmap          # Feature matrix
    y_memmap: np.memmap          # Target vector
    chunk_size: int              # For batch processing

FUNCTION get_batch(start_idx, end_idx):
    # Lazy loading of data chunks
    X_batch = self.X_memmap[start_idx:end_idx]
    y_batch = self.y_memmap[start_idx:end_idx]
    RETURN X_batch, y_batch

FUNCTION parallel_bootstrap_sample(n_samples, n_trees, n_workers):
    # Generate all bootstrap indices in parallel
    with PARALLEL_POOL(n_workers):
        bootstrap_indices = PARALLEL_MAP(
            lambda _: np.random.choice(n_samples, n_samples, replace=True),
            range(n_trees)
        )
    RETURN bootstrap_indices
```

### Performance Profiling

#### 1. Timing Critical Paths

```
DECORATOR: @profile_time
FUNCTION profile_time(func):
    FUNCTION wrapper(*args, **kwargs):
        start_time = TIME()
        result = func(*args, **kwargs)
        end_time = TIME()
        
        PROFILER.record(func.__name__, end_time - start_time)
        RETURN result
    
    RETURN wrapper

CLASS RandomForestProfiler:
    FIELDS:
        timings: Dict[str, List[float]]
    
    @profile_time
    FUNCTION fit(self, X, y):
        # All fit operations are automatically timed
        pass
    
    @profile_time  
    FUNCTION find_best_split(self, node_data):
        # Split finding timing
        pass
        
    FUNCTION get_performance_report():
        report = {}
        FOR operation, times in self.timings.items():
            report[operation] = {
                'total_time': sum(times),
                'average_time': mean(times),
                'max_time': max(times),
                'call_count': len(times)
            }
        RETURN report
```

#### 2. Memory Usage Tracking

```
FUNCTION estimate_memory_usage(n_samples, n_features, n_trees, max_depth):
    # Estimate memory requirements
    
    # Data storage
    data_memory = n_samples * n_features * 4  # 4 bytes per float32
    
    # Tree storage (compact representation)
    max_nodes_per_tree = 2**(max_depth + 1) - 1
    nodes_memory_per_tree = max_nodes_per_tree * (
        4 +    # feature_index (int16) + padding
        4 +    # threshold (float32)  
        4 +    # left_child (int32)
        4 +    # right_child (int32)
        4 +    # leaf_value (float32)
        1      # is_leaf (bool)
    )
    
    total_tree_memory = n_trees * nodes_memory_per_tree
    
    # Bootstrap indices storage  
    bootstrap_memory = n_trees * n_samples * 4  # int32 indices
    
    # OOB tracking
    oob_memory = n_samples * (4 + 4)  # predictions + counts
    
    total_memory = data_memory + total_tree_memory + bootstrap_memory + oob_memory
    
    RETURN {
        'data_mb': data_memory / 1024**2,
        'trees_mb': total_tree_memory / 1024**2, 
        'bootstrap_mb': bootstrap_memory / 1024**2,
        'oob_mb': oob_memory / 1024**2,
        'total_mb': total_memory / 1024**2
    }
```

### Testing and Validation

#### 1. Unit Tests for Core Components

```
FUNCTION test_bootstrap_sampling():
    n_samples = 1000
    sampler = BootstrapSampler()
    
    # Test sample size
    indices = sampler.sample(n_samples)
    ASSERT len(indices) == n_samples
    
    # Test with replacement (should have duplicates)
    unique_indices = set(indices)
    ASSERT len(unique_indices) < n_samples
    
    # Test coverage (should cover ~63.2% of samples)
    coverage = len(unique_indices) / n_samples
    ASSERT 0.6 <= coverage <= 0.67

FUNCTION test_feature_importance_consistency():
    # Generate synthetic data with known feature importance
    X = GENERATE_SYNTHETIC_DATA(n_samples=1000, n_features=10, random_state=42)
    y = X[:, 0] + 0.5 * X[:, 1] + NOISE()  # Features 0,1 important, rest noise
    
    rf = RandomForest(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    importance = rf.feature_importances_
    
    # Most important features should be 0 and 1
    top_features = ARGSORT(importance)[-2:]
    ASSERT 0 in top_features
    ASSERT 1 in top_features
    
    # Sum should be 1
    ASSERT ABS(SUM(importance) - 1.0) < 1e-10

FUNCTION test_oob_score_validity():
    X, y = LOAD_DATASET('classification')
    rf = RandomForest(n_estimators=100)
    rf.fit(X, y)
    
    oob_score = rf.oob_score_
    
    # OOB score should be reasonable (0.7-1.0 for good datasets)
    ASSERT 0.0 <= oob_score <= 1.0
    
    # Compare with cross-validation
    cv_scores = CROSS_VAL_SCORE(rf, X, y, cv=5)
    cv_mean = MEAN(cv_scores)
    
    # OOB and CV should be similar (within reasonable range)
    ASSERT ABS(oob_score - cv_mean) < 0.1
```

#### 2. Integration Tests

```
FUNCTION test_end_to_end_regression():
    # Complete pipeline test
    X, y = MAKE_REGRESSION(n_samples=1000, n_features=20, noise=0.1, random_state=42)
    
    # Train-test split
    X_train, X_test, y_train, y_test = TRAIN_TEST_SPLIT(X, y, test_size=0.2)
    
    # Fit model
    rf = RandomForest(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    
    # Predictions
    y_pred = rf.predict(X_test)
    
    # Performance metrics
    mse = MEAN_SQUARED_ERROR(y_test, y_pred)
    r2 = R2_SCORE(y_test, y_pred)
    
    # Sanity checks
    ASSERT mse > 0
    ASSERT r2 > 0.8  # Should achieve good performance on synthetic data
    ASSERT len(y_pred) == len(y_test)
    
    # Feature importance
    importance = rf.feature_importances_
    ASSERT len(importance) == X.shape[1]
    ASSERT ABS(SUM(importance) - 1.0) < 1e-10

FUNCTION test_memory_efficiency():
    # Test memory usage doesn't explode with large datasets
    n_samples_large = 50000
    n_features = 100
    
    X = np.random.randn(n_samples_large, n_features).astype(np.float32)
    y = np.random.randn(n_samples_large).astype(np.float32)
    
    initial_memory = GET_MEMORY_USAGE()
    
    rf = RandomForest(n_estimators=10, max_depth=8)  # Small forest to control memory
    rf.fit(X, y)
    
    final_memory = GET_MEMORY_USAGE()
    memory_increase = final_memory - initial_memory
    
    # Should not use excessive memory (heuristic: < 10x data size)
    data_size = X.nbytes + y.nbytes
    ASSERT memory_increase < 10 * data_size
```

### Configuration Management

#### 1. Hierarchical Configuration

```
CONFIGURATION: RandomForestConfig
defaults:
  ensemble:
    n_estimators: 100
    bootstrap: true
    oob_score: true
    
  tree:
    criterion: "gini"  # or "entropy", "mse", "mae"
    max_depth: null
    min_samples_split: 2
    min_samples_leaf: 1
    min_impurity_decrease: 0.0
    
  features:
    max_features: "sqrt"  # or "log2", int, float
    feature_sampling: "without_replacement"
    
  performance:
    n_jobs: -1
    random_state: null
    verbose: 0
    
  advanced:
    class_weight: null
    ccp_alpha: 0.0
    max_samples: null

FUNCTION load_config(config_path, overrides=None):
    base_config = LOAD_YAML(config_path)
    
    IF overrides:
        config = DEEP_MERGE(base_config, overrides)
    ELSE:
        config = base_config
    
    RETURN validate_config(config)

FUNCTION validate_config(config):
    # Type checking and constraint validation
    IF config.ensemble.n_estimators <= 0:
        RAISE ValueError("n_estimators must be positive")
    
    IF config.tree.max_depth is not None AND config.tree.max_depth <= 0:
        RAISE ValueError("max_depth must be positive or None")
    
    # Feature sampling validation
    max_features = config.features.max_features
    IF isinstance(max_features, str):
        ASSERT max_features in ["sqrt", "log2"]
    ELIF isinstance(max_features, int):
        ASSERT max_features > 0
    ELIF isinstance(max_features, float):
        ASSERT 0.0 < max_features <= 1.0
    
    RETURN config
```

#### 2. Experiment Tracking Integration

```
CLASS ExperimentTracker:
    FUNCTION __init__(self, experiment_name, backend="mlflow"):
        self.experiment_name = experiment_name
        self.backend = backend
        self.run_id = GENERATE_RUN_ID()
    
    FUNCTION log_config(self, config):
        FOR key, value in FLATTEN_DICT(config).items():
            self.backend.log_param(key, value)
    
    FUNCTION log_metrics(self, metrics_dict, step=None):
        FOR metric_name, metric_value in metrics_dict.items():
            self.backend.log_metric(metric_name, metric_value, step)
    
    FUNCTION log_feature_importance(self, feature_names, importance_values):
        importance_dict = dict(zip(feature_names, importance_values))
        self.backend.log_dict(importance_dict, "feature_importance.json")
    
    FUNCTION log_model(self, model, model_name="random_forest"):
        self.backend.log_model(model, model_name)

# Usage example
tracker = ExperimentTracker("random_forest_optimization")
tracker.log_config(config)

rf = RandomForest(**config)
rf.fit(X_train, y_train)

metrics = {
    "oob_score": rf.oob_score_,
    "test_accuracy": ACCURACY_SCORE(y_test, rf.predict(X_test)),
    "training_time": training_time,
    "n_nodes_total": sum(tree.get_n_leaves() for tree in rf.estimators_)
}

tracker.log_metrics(metrics)
tracker.log_feature_importance(feature_names, rf.feature_importances_)
tracker.log_model(rf)
```

---

## Conclusion and Best Practices

### Algorithm Selection Guidelines

**Decision Framework:**
```
Problem Type Assessment:
├─ Tabular data, moderate size → Random Forest
├─ Very large dataset → ExtraTrees or Subbagging  
├─ High-dimensional, sparse → Random Subspaces + Feature Hashing
├─ Imbalanced classes → Balanced Random Forest
├─ Streaming data → Online Random Forest
├─ Anomaly detection → Isolation Forest
├─ Survival analysis → Random Survival Forest
└─ Causal inference → Causal Forest

Performance Requirements:
├─ Maximum accuracy → Random Forest with hyperparameter tuning
├─ Speed priority → ExtraTrees with fewer estimators
├─ Memory constraints → Subbagging with compact trees
└─ Interpretability → Fewer, deeper trees with feature importance
```

### Hyperparameter Tuning Strategy

**Progressive Tuning Approach:**
1. **Start with defaults:** Get baseline performance
2. **Tune ensemble size:** Find optimal n_estimators via OOB score
3. **Tune tree complexity:** max_depth, min_samples_split, min_samples_leaf
4. **Tune feature sampling:** m_try parameter
5. **Advanced tuning:** class_weight, bootstrap strategy

**Recommended Grid Search:**
```
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.3, 0.5]
}
```

### Common Pitfalls and Solutions

**Overfitting Prevention:**
- Use OOB score for early stopping
- Limit tree depth and increase min_samples_leaf
- Enable bootstrap (don't use full dataset per tree)
- Regularize with min_impurity_decrease

**Performance Optimization:**
- Use feature selection to reduce dimensionality
- Enable parallel processing (n_jobs=-1)
- Consider subsampling for very large datasets
- Use memory-mapped arrays for huge datasets

**Interpretation Best Practices:**
- Use permutation importance over MDI when possible
- Report confidence intervals for importance scores
- Visualize partial dependence plots for top features
- Use SHAP for instance-level explanations